In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import random

plt.rcParams['savefig.dpi'] = 144
%matplotlib inline

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

## **Làm sạch dữ liệu**

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/web_crawler/data/car_raw.csv")
df.head()

,Price,CarBrand,CarModel,MfgDate,Mileage,Condition,GearBox,Fuel,CarOrigin,CarType,CarSeats,WarrantyPolicy
0,940.000.000 đ,Vinfast,VF8,2023,NaN,Mới,Tự động,Điện,Việt Nam,SUV / Cross over,5,Bảo hành hãng
1,999.000.000 đ,Hyundai,Santa Fe,2021,20000.0,Đã sử dụng,Tự động,Xăng,Việt Nam,SUV / Cross over,7,Bảo hành hãng
2,1.058.000.000 đ,Hyundai,Santa Fe,2021,24000.0,Đã sử dụng,Tự động,Dầu,Đang cập nhật,SUV / Cross over,7,Bảo hành hãng
3,460.000.000 đ,Mazda,3,2018,44658.0,Đã sử dụng,Tự động,Xăng,Việt Nam,Sedan,5,Bảo hành hãng
4,415.000.000 đ,Lexus,GS,2007,135000.0,Đã sử dụng,Tự động,Xăng,Mỹ,Sedan,4,Bảo hành hãng


**Mô tả thông tin dữ liệu, gồm có 12 đặc trưng như sau:**
1. *Price*: giá của xe

2. *CarBrand*: hãng xe

3. *CarModel*: dòng xe

4. *MfgDate*: năm sản xuất

5. *Mileage*: số km đã đi

6. *Condition*: trạng thái sử dụng

7. *GearBox*: loại hộp số

8. *Fuel*: loại nhiên liệu

9. *CarOrigin*: xuất xứ

10. *CarType*: phân khúc

11. *CarSeats*: số chỗ ngồi

12. *WarrantyPolicy*: loại bảo hành

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15382 entries, 0 to 15381
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Price           14030 non-null  object 
 1   CarBrand        14030 non-null  object 
 2   CarModel        14030 non-null  object 
 3   MfgDate         14031 non-null  object 
 4   Mileage         11894 non-null  float64
 5   Condition       14035 non-null  object 
 6   GearBox         14030 non-null  object 
 7   Fuel            14029 non-null  object 
 8   CarOrigin       14035 non-null  object 
 9   CarType         12056 non-null  object 
 10  CarSeats        12551 non-null  object 
 11  WarrantyPolicy  14034 non-null  object 
dtypes: float64(1), object(11)
memory usage: 1.4+ MB


In [ ]:
df.columns

Index(['Price', 'CarBrand', 'CarModel', 'MfgDate', 'Mileage', 'Condition',
       'GearBox', 'Fuel', 'CarOrigin', 'CarType', 'CarSeats',
       'WarrantyPolicy'],
      dtype='object')

In [ ]:
# chỉnh sửa lại giá và đưa về kiểu dữ liệu số
df['Price'] = df["Price"].str.replace(".", "", regex=True)
df['Price'] = df['Price'].str.replace(" đ", "")
df['Price'] = df['Price'].astype(float)
df.head()

,Price,CarBrand,CarModel,MfgDate,Mileage,Condition,GearBox,Fuel,CarOrigin,CarType,CarSeats,WarrantyPolicy
0,9.400000e+08,Vinfast,VF8,2023,NaN,Mới,Tự động,Điện,Việt Nam,SUV / Cross over,5,Bảo hành hãng
1,9.990000e+08,Hyundai,Santa Fe,2021,20000.0,Đã sử dụng,Tự động,Xăng,Việt Nam,SUV / Cross over,7,Bảo hành hãng
2,1.058000e+09,Hyundai,Santa Fe,2021,24000.0,Đã sử dụng,Tự động,Dầu,Đang cập nhật,SUV / Cross over,7,Bảo hành hãng
3,4.600000e+08,Mazda,3,2018,44658.0,Đã sử dụng,Tự động,Xăng,Việt Nam,Sedan,5,Bảo hành hãng
4,4.150000e+08,Lexus,GS,2007,135000.0,Đã sử dụng,Tự động,Xăng,Mỹ,Sedan,4,Bảo hành hãng


In [ ]:
# nếu xe là xe mới, nên số km đi được là 0
# đưa dữ liệu về kiểu số
df['Mileage'] = df.apply(lambda row: 0 if pd.isnull(row['Mileage']) and row['Condition'] == 'Mới' else row['Mileage'], axis=1)
df['Mileage'] = pd.to_numeric(df['Mileage'], errors='coerce')

In [ ]:
# thay thế tất cả xuất xứ của xe 'Đang cập nhật' thành None
df['CarOrigin'] = df['CarOrigin'].replace('Đang cập nhật', np.nan)
df['CarOrigin'].unique()

array(['Việt Nam', nan, 'Mỹ', 'Thái Lan', 'Nhật Bản', 'Nước khác',
       'Hàn Quốc', 'Đức', 'Đài Loan', 'Ấn Độ', 'Trung Quốc'], dtype=object)

In [ ]:
# thay thế 'Hãng khác' = None
df['CarBrand'] = df['CarBrand'].replace('Hãng khác', np.nan)
df['CarBrand'].unique()


array(['Vinfast', 'Hyundai', 'Mazda', 'Lexus', 'Mitsubishi', 'Toyota',
       'Mercedes Benz', 'Ford', 'Honda', 'Isuzu', 'Peugeot', 'BMW',
       'Daewoo', 'Kia', 'Volkswagen', 'Suzuki', 'Porsche', nan, 'Nissan',
       'Chevrolet', 'Mini', 'Jaguar', 'Audi', 'Bentley', 'Subaru', 'MG',
       'Cadillac', 'Volvo', 'Proton', 'Baic', 'Haval', 'Daihatsu',
       'Lifan', 'LandRover', 'Smart', 'Ferrari', 'Jeep', 'Renault',
       'Fiat', 'Acura', 'Luxgen', 'Haima', 'SYM', 'Maserati', 'Infiniti',
       'UAZ', 'Ssangyong', 'Asia', 'Chrysler', 'Dodge', 'Rover', 'HongQi',
       'Mekong', 'Chery', 'Rolls Royce', 'Zotye', 'RAM', 'Citroen', 'Gaz'],
      dtype=object)

In [ ]:
# thay thế 'trước năm 1980' thành none
# và đưa các giá trị về kiểu số
df['MfgDate'] = df['MfgDate'].replace('trước năm 1980', np.nan)
df['MfgDate'] = pd.to_numeric(df['MfgDate'], errors='coerce')
df['MfgDate'].unique()

array([2023., 2021., 2018., 2007., 2019., 2022., 2017., 2011., 2010.,
       2020., 2015., 2008., 2012., 1998., 2003., 2016.,   nan, 2000.,
       1997., 1995., 2014., 2005., 2004., 2013., 2009., 2006., 2002.,
       2001., 1990., 1991., 1992., 1994., 1996., 1985., 1993., 1989.,
       1983., 1987., 1999., 1986., 1984., 1988.])

In [ ]:
# thay thế các giá trị 5 bằng none
df['GearBox'] = df['GearBox'].replace('5', np.nan)
df['GearBox'] = df['GearBox'].replace('4', np.nan)
df['GearBox'].unique()

array(['Tự động', 'Số sàn', 'Bán tự động', nan], dtype=object)

In [ ]:
df['Condition'].unique()

array(['Mới', 'Đã sử dụng', nan], dtype=object)

In [ ]:
# chỉnh sửa 'Động cơ Hybrid' thành 'Hybrid'
df['Fuel'] = df['Fuel'].replace('Động cơ Hybrid', 'Hybrid')
df['Fuel'].unique()

array(['Điện', 'Xăng', 'Dầu', 'Hybrid', nan], dtype=object)

In [ ]:
# thay thế các giá trị
df['CarType'] = df['CarType'].replace('Kiểu dáng khác', np.nan)
df['CarType'] = df['CarType'].replace('SUV / Cross over', 'SUV')
df['CarType'] = df['CarType'].replace('Minivan (MPV)', 'MPV')
df['CarType'] = df['CarType'].replace('Pick-up (bán tải)', 'Pick up')
df['CarType'] = df['CarType'].replace('Coupe (2 cửa)', 'Coupe')
df['CarType'].unique()

array(['SUV', 'Sedan', 'MPV', 'Van', nan, 'Pick up', 'Coupe', 'Hatchback',
       'Mui trần'], dtype=object)

In [ ]:
# thay thế 'Khác' thành none, và đưa dữ liệu về kiểu số
df['CarSeats'] = df['CarSeats'].replace('Khác', np.nan)
df['CarSeats'] = pd.to_numeric(df['CarSeats'], errors='coerce')
df['CarSeats'].unique()

array([ 5.,  7.,  4.,  9., 16., nan,  2.,  8.,  6., 10., 12.])

In [ ]:
df['WarrantyPolicy'].unique()

array(['Bảo hành hãng', nan], dtype=object)

In [ ]:
# xoá các mẫu, có nhiều hơn 2 giá trị trống
df.dropna(thresh=len(df.columns)-2, inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12869 entries, 0 to 15378
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Price           12866 non-null  float64
 1   CarBrand        12817 non-null  object 
 2   CarModel        12868 non-null  object 
 3   MfgDate         12851 non-null  float64
 4   Mileage         12734 non-null  float64
 5   Condition       12869 non-null  object 
 6   GearBox         12814 non-null  object 
 7   Fuel            12869 non-null  object 
 8   CarOrigin       10258 non-null  object 
 9   CarType         11898 non-null  object 
 10  CarSeats        12506 non-null  float64
 11  WarrantyPolicy  12869 non-null  object 
dtypes: float64(4), object(8)
memory usage: 1.3+ MB


In [ ]:
## lưu dữ liệu đã làm sạch

df.to_csv('/content/drive/MyDrive/web_crawler/data/clean_car.csv', index=False, encoding='utf-8')